In [1]:
!pip install requests
!pip install tabulate
!pip install 'colorama>=0.3.8'
!pip install future

In [2]:
!pip install h2o

     |████████████████████████████████| 175.8 MB 36 kB/s 
  Created wheel for h2o: filename=h2o-3.34.0.7-py2.py3-none-any.whl size=175908084 sha256=dc4f9b3d35b583eddfdfcb577d95eafeaa4e4e72f56a8529057c659fcdde0f89
  Stored in directory: /root/.cache/pip/wheels/df/df/05/1f2ac68a5d9bf38b74f2b8218443b2c00f1a5f4f3154ec11b6
Successfully built h2o


In [3]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(max_mem_size='16G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.13" 2021-10-19; OpenJDK Runtime Environment (build 11.0.13+8-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.13+8-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp6y3i8lz5
  JVM stdout: /tmp/tmp6y3i8lz5/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp6y3i8lz5/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.7
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_unknownUser_9nq6r8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,16 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
churn_data=h2o.import_file('/content/drive/MyDrive/Colab Notebooks/Customer+churn+files.zip (Unzipped Files)/Notebook/Churn Modeling.csv')

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [5]:
churn_data.head()

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,1.56346e+07,Hargrave,619,France,Female,42,2,0,1,1,1,101349,1
2,1.56473e+07,Hill,608,Spain,Female,41,1,83807.9,1,0,1,112543,0
3,1.56193e+07,Onio,502,France,Female,42,8,159661,3,1,0,113932,1
4,1.57014e+07,Boni,699,France,Female,39,1,0,2,0,0,93826.6,0
5,1.57379e+07,Mitchell,850,Spain,Female,43,2,125511,1,1,1,79084.1,0
6,1.5574e+07,Chu,645,Spain,Male,44,8,113756,2,1,0,149757,1
7,1.55925e+07,Bartlett,822,France,Male,50,7,0,2,1,1,10062.8,0
8,1.56561e+07,Obinna,376,Germany,Female,29,4,115047,4,1,0,119347,1
9,1.57924e+07,He,501,France,Male,44,4,142051,2,0,1,74940.5,0
10,1.55924e+07,H?,684,France,Male,27,2,134604,1,1,1,71725.7,0


In [6]:
churn_data_train,churn_data_test=churn_data.split_frame(ratios=[0.8])

In [7]:
churn_data_train

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,1.56346e+07,Hargrave,619,France,Female,42,2,0,1,1,1,101349,1
3,1.56193e+07,Onio,502,France,Female,42,8,159661,3,1,0,113932,1
4,1.57014e+07,Boni,699,France,Female,39,1,0,2,0,0,93826.6,0
5,1.57379e+07,Mitchell,850,Spain,Female,43,2,125511,1,1,1,79084.1,0
6,1.5574e+07,Chu,645,Spain,Male,44,8,113756,2,1,0,149757,1
7,1.55925e+07,Bartlett,822,France,Male,50,7,0,2,1,1,10062.8,0
9,1.57924e+07,He,501,France,Male,44,4,142051,2,0,1,74940.5,0
10,1.55924e+07,H?,684,France,Male,27,2,134604,1,1,1,71725.7,0
11,1.57678e+07,Bearce,528,France,Male,31,6,102017,2,0,0,80181.1,0
12,1.57372e+07,Andrews,497,Spain,Male,24,3,0,2,1,0,76390,0


In [8]:
y='Exited'
x=churn_data.columns
x.remove(y)
x.remove('CustomerId')
x.remove('Surname')
x.remove('RowNumber')

In [9]:
aml=H2OAutoML(max_runtime_secs=300,max_models=10,verbosity='info',nfolds=0,seed=10)

In [10]:
aml.train(x=x,y=y,training_frame=churn_data_train)

AutoML progress: |
07:43:26.268: Project: AutoML_1_20220123_74326
07:43:26.270: Cross-validation disabled by user: no fold column nor nfolds > 1.
07:43:26.278: Setting stopping tolerance adaptively based on the training frame: 0.01115318704636506
07:43:26.279: Build control seed: 10
07:43:26.280: Since cross-validation is disabled, and none of validation frame and leaderboard frame were provided, automatically split the training data into training, validation and leaderboard frames in the ratio 80/10/10
07:43:26.378: training frame: Frame key: AutoML_1_20220123_74326_training_py_3_sid_bcba    cols: 14    rows: 6413  chunks: 8    size: 448804  checksum: 3084066995262909508
07:43:26.393: validation frame: Frame key: AutoML_1_20220123_74326_validation_py_3_sid_bcba    cols: 14    rows: 830  chunks: 8    size: 278518  checksum: -1594256810509285939
07:43:26.404: leaderboard frame: Frame key: AutoML_1_20220123_74326_leaderboard_py_3_sid_bcba    cols: 14    rows: 796  chunks: 8    size: 2774

,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,52869.0,8.0,8.0,8.0,28.0,132.0,79.48




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.06815545951018244
RMSE: 0.2610660060409674
MAE: 0.16758945433980185
RMSLE: 0.1815700411325247
Mean Residual Deviance: 0.06815545951018244

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 0.10500871345544817
RMSE: 0.3240504797951211
MAE: 0.213855536072082
RMSLE: 0.2273680505396469
Mean Residual Deviance: 0.10500871345544817

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-01-23 07:43:37,0.004 sec,0.0,0.402826,0.324537,0.162268,0.407968,0.328706,0.166438
1,,2022-01-23 07:43:37,0.131 sec,5.0,0.345853,0.273271,0.119614,0.360291,0.285342,0.129810
2,,2022-01-23 07:43:37,0.241 sec,10.0,0.314247,0.237559,0.098751,0.339153,0.257259,0.115025
3,,2022-01-23 07:43:38,0.352 sec,15.0,0.297889,0.217573,0.088738,0.331130,0.243173,0.109647
4,,2022-01-23 07:43:38,0.450 sec,20.0,0.286288,0.199960,0.081961,0.327765,0.231021,0.107430
5,,2022-01-23 07:43:38,0.555 sec,25.0,0.278911,0.189195,0.077792,0.326214,0.224881,0.106416
6,,2022-01-23 07:43:38,0.656 sec,30.0,0.274406,0.181947,0.075298,0.325248,0.220159,0.105786
7,,2022-01-23 07:43:38,0.761 sec,35.0,0.270128,0.176703,0.072969,0.324704,0.217243,0.105433
8,,2022-01-23 07:43:38,0.855 sec,40.0,0.266839,0.172455,0.071203,0.323776,0.214639,0.104831
9,,2022-01-23 07:43:38,0.940 sec,45.0,0.263615,0.169480,0.069493,0.324370,0.214132,0.105216



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Age,910.996521,1.000000,0.305357
1,NumOfProducts,572.487671,0.628419,0.191892
2,Balance,418.957825,0.459890,0.140431
3,EstimatedSalary,268.025299,0.294211,0.089839
4,IsActiveMember,211.602966,0.232276,0.070927
5,CreditScore,209.963486,0.230477,0.070378
6,Tenure,161.954056,0.177777,0.054285
7,Geography,138.045349,0.151532,0.046271
8,Gender,57.177696,0.062764,0.019165
9,HasCrCard,34.168659,0.037507,0.011453


In [11]:
lb=aml.leaderboard

In [12]:
lb

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
GBM_3_AutoML_1_20220123_74326,0.101512,0.31861,0.101512,0.205341,0.223632
GBM_1_AutoML_1_20220123_74326,0.101944,0.319287,0.101944,0.20942,0.223568
XGBoost_3_AutoML_1_20220123_74326,0.10372,0.322056,0.10372,0.210321,0.225948
GBM_2_AutoML_1_20220123_74326,0.103829,0.322225,0.103829,0.209144,0.225621
DRF_1_AutoML_1_20220123_74326,0.104751,0.323652,0.104751,0.211067,0.228207
GBM_4_AutoML_1_20220123_74326,0.105479,0.324775,0.105479,0.209494,0.228443
XRT_1_AutoML_1_20220123_74326,0.111336,0.33367,0.111336,0.241538,0.237706
XGBoost_2_AutoML_1_20220123_74326,0.121075,0.347958,0.121075,0.232958,0.249895
XGBoost_1_AutoML_1_20220123_74326,0.121513,0.348587,0.121513,0.240582,0.252577
GLM_1_AutoML_1_20220123_74326,0.140571,0.374927,0.140571,0.286445,0.264489


In [13]:
model_ids=list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])

In [14]:
model_ids

['GBM_3_AutoML_1_20220123_74326',
 'GBM_1_AutoML_1_20220123_74326',
 'XGBoost_3_AutoML_1_20220123_74326',
 'GBM_2_AutoML_1_20220123_74326',
 'DRF_1_AutoML_1_20220123_74326',
 'GBM_4_AutoML_1_20220123_74326',
 'XRT_1_AutoML_1_20220123_74326',
 'XGBoost_2_AutoML_1_20220123_74326',
 'XGBoost_1_AutoML_1_20220123_74326',
 'GLM_1_AutoML_1_20220123_74326']

In [15]:
aml.leader.model_performance(churn_data_test)


ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 0.10109345569169619
RMSE: 0.31795197073095205
MAE: 0.2057116880944887
RMSLE: 0.22405332999806785
Mean Residual Deviance: 0.10109345569169619


In [16]:
models=h2o.get_model([mid for mid in model_ids if 'GBM' in  mid][0])

In [17]:
models

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_3_AutoML_1_20220123_74326


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,50.0,50.0,52869.0,8.0,8.0,8.0,28.0,132.0,79.48




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 0.06815545951018244
RMSE: 0.2610660060409674
MAE: 0.16758945433980185
RMSLE: 0.1815700411325247
Mean Residual Deviance: 0.06815545951018244

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 0.10500871345544817
RMSE: 0.3240504797951211
MAE: 0.213855536072082
RMSLE: 0.2273680505396469
Mean Residual Deviance: 0.10500871345544817

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-01-23 07:43:37,0.004 sec,0.0,0.402826,0.324537,0.162268,0.407968,0.328706,0.166438
1,,2022-01-23 07:43:37,0.131 sec,5.0,0.345853,0.273271,0.119614,0.360291,0.285342,0.129810
2,,2022-01-23 07:43:37,0.241 sec,10.0,0.314247,0.237559,0.098751,0.339153,0.257259,0.115025
3,,2022-01-23 07:43:38,0.352 sec,15.0,0.297889,0.217573,0.088738,0.331130,0.243173,0.109647
4,,2022-01-23 07:43:38,0.450 sec,20.0,0.286288,0.199960,0.081961,0.327765,0.231021,0.107430
5,,2022-01-23 07:43:38,0.555 sec,25.0,0.278911,0.189195,0.077792,0.326214,0.224881,0.106416
6,,2022-01-23 07:43:38,0.656 sec,30.0,0.274406,0.181947,0.075298,0.325248,0.220159,0.105786
7,,2022-01-23 07:43:38,0.761 sec,35.0,0.270128,0.176703,0.072969,0.324704,0.217243,0.105433
8,,2022-01-23 07:43:38,0.855 sec,40.0,0.266839,0.172455,0.071203,0.323776,0.214639,0.104831
9,,2022-01-23 07:43:38,0.940 sec,45.0,0.263615,0.169480,0.069493,0.324370,0.214132,0.105216



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Age,910.996521,1.000000,0.305357
1,NumOfProducts,572.487671,0.628419,0.191892
2,Balance,418.957825,0.459890,0.140431
3,EstimatedSalary,268.025299,0.294211,0.089839
4,IsActiveMember,211.602966,0.232276,0.070927
5,CreditScore,209.963486,0.230477,0.070378
6,Tenure,161.954056,0.177777,0.054285
7,Geography,138.045349,0.151532,0.046271
8,Gender,57.177696,0.062764,0.019165
9,HasCrCard,34.168659,0.037507,0.011453


In [18]:
models.params

{'auc_type': {'actual': 'AUTO', 'default': 'AUTO', 'input': 'AUTO'},
 'balance_classes': {'actual': False, 'default': False, 'input': False},
 'build_tree_one_node': {'actual': False, 'default': False, 'input': False},
 'calibrate_model': {'actual': False, 'default': False, 'input': False},
 'calibration_frame': {'actual': None, 'default': None, 'input': None},
 'categorical_encoding': {'actual': 'Enum',
  'default': 'AUTO',
  'input': 'AUTO'},
 'check_constant_response': {'actual': True, 'default': True, 'input': True},
 'checkpoint': {'actual': None, 'default': None, 'input': None},
 'class_sampling_factors': {'actual': None, 'default': None, 'input': None},
 'col_sample_rate': {'actual': 0.8, 'default': 1.0, 'input': 0.8},
 'col_sample_rate_change_per_level': {'actual': 1.0,
  'default': 1.0,
  'input': 1.0},
 'col_sample_rate_per_tree': {'actual': 0.8, 'default': 1.0, 'input': 0.8},
 'custom_distribution_func': {'actual': None, 'default': None, 'input': None},
 'custom_metric_func'

In [19]:
y_preds=aml.leader.predict(churn_data_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [33]:
y_preds

predict
0.237336
1.1436
0.181587
0.0416541
0.450554
0.0228652
0.370057
0.370642
0.00967723
0.0478304
